In [1]:
import ast
import pickle
import os
import pandas as pd

#### Luigi migration. Task yet to be created

In [25]:
import os
import sys
module_path = os.path.abspath(os.path.join("../../.."))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from jupyter_notebook import load_parameters 


pars = load_parameters()

so_dump_processed_file = pars.get('input')
output_file = pars.get('output')

input_col = pars.get('input_col')

api_doc_file = pars.get('api_doc_file')

debug = pars.get('debug')

KeyError: 'PARS'

#### Utility to extract method name

In [2]:
from collections import deque

class FunctionCallVisitor(ast.NodeVisitor):
    def __init__(self):
        self._name = deque()
        self._pos = -1 
    
    @property
    def name(self):
        return '.'.join(self._name)
    
    @property
    def lineno(self):
        return self._pos
    
    @name.deleter
    def name(self):
        self._name.clear()
    
    def visit_Name(self, node):
        self._pos = node.lineno # line number
        self._name.appendleft(node.id)
    
    def visit_Attribute(self, node):
        try:
            self._pos = node.lineno # line number
            self._name.appendleft(node.attr)
            self._name.appendleft(node.value.id)
        except AttributeError:
            self.generic_visit(node)
            
def get_func_calls(tree):
    func_calls = []
    for node in ast.walk(tree):
        if isinstance(node, ast.Call):
            callvisitor = FunctionCallVisitor()
            callvisitor.visit(node.func)
            func_calls.append((callvisitor.name, callvisitor.lineno))
    return func_calls

#### Load APIDoc and Dataset

In [3]:
api_doc_file = '../../data-import/build_api_doc_base/api_doc.csv'
# tagged_dataset_file = '../../../data/stack-overflow/Dataset - Pandas.csv'
input_file = '../../../data/stack-overflow/pandas-preprocessedcode-dataset-part3'

api_df = pd.read_csv(api_doc_file, encoding='ISO-8859-1', error_bad_lines=False)
# tagged_dataset_df = pd.read_csv(tagged_dataset_file, encoding='ISO-8859-1', error_bad_lines=False)
processed_stackoverflow_df = pd.read_pickle(input_file)
input_col = "PreprocessedCode3"

def buildAPIDictionary(api_df):
    api_dict = dict()
    try:
        
        for index, row in api_df.iterrows():
            methodContext = row['SubCategory']
            #tokens = row['FullyQualifiedName'].split('.')
        
            #for token in tokens:
               #methodContext = str(methodContext)+' '+token
            api_dict[row['MethodName']] = methodContext
    except Exception as e:
        print('Error in method buildAPIDictionary',e)
    return api_dict
        
def buildStackOverflowDumpDict(processed_stackoverflow_df):
    stackoverflow_dict = dict()
    try:
        for idx, row in processed_stackoverflow_df.iterrows():
            postTypeId = row['PostTypeId']
            if postTypeId == 2:
                answerId = row['Id']
                stackoverflow_dict[answerId] = row['PreprocessedCode']
    except Exception as e:
        print('Error in method buildStackOverflowDumpDict', e)
    return stackoverflow_dict
    
api_dict = buildAPIDictionary(api_df)
# tagged_dataset_dict, total_solutions = buildTaggedDatasetDSForEvaluation(tagged_dataset_df)
stackoverflow_dict = buildStackOverflowDumpDict(processed_stackoverflow_df)

In [4]:
def lookUpAPIDocForContextH2(method_name):
    try:
        if method_name in api_dict.keys() and api_dict[method_name] == "Constructor":
            return True
        else:
            return False
    except Exception as e:
        print('Error in method lookUpAPIDocForContext', e)
        

In [8]:
def H2(df):
    
    # Do not process questions
    if df.PostTypeId == 1:
        df['Solution'] = 'NA'
    
    else:
        # Parse code and inspect the function
        code_snippet = df[input_col]
        tree = ast.parse(code_snippet)        
        func_calls = get_func_calls(tree)
        
        snippet_per_line = code_snippet.split(os.linesep)

        # Start with all the lines
        solution_lines = set(range(len(snippet_per_line)))
        
        for function, lineno in func_calls:   
            # print(function)
            tokens = function.split('.')
            method_name = tokens[len(tokens)-1]
            
            # Check whether the func_call is a constructor to remove it
            if lookUpAPIDocForContextH2(method_name):
                # Remove the lines that ARE a constructor
                idx = lineno - 1
                # Check whether we have remove it before to avoid any issue
                if idx in solution_lines:
                    solution_lines.remove(idx)
                
        solution = []
        solution_lines = sorted(solution_lines)
        for i in solution_lines:
            solution.append(snippet_per_line[i])
                
        #print(solution)
        df['Solution'] = os.linesep.join(solution)
    return df


solution_df = processed_stackoverflow_df.apply(H2, axis=1)

,Unnamed: 0,Unnamed: 0.1,AcceptedAnswerId,AnswerCount,Body,ClosedDate,CommentCount,CommunityOwnedDate,CreationDate,FavoriteCount,...,PostTypeId,Score,Tags,Title,ViewCount,Code,PreprocessedCode,PreprocessedCode2,PreprocessedCode3,Solution
0,27,27,7779260.0,2.0,<p>When I try to merge two dataframes by rows ...,NaN,7,NaN,2011-10-15T08:21:17.460,4.0,...,1,25,<python><pandas>,append two data frame with pandas,25479.0,bigdata = data1.append(data2)\r\nException: In...,bigdata = data1.append(data2)\r\nException: In...,bigdata = data1.append(data2)\r\nException: In...,bigdata = data1.append(data2),NA
1,31,31,11617194.0,7.0,<p>I want to perform my own complex operations...,NaN,3,NaN,2011-10-20T14:46:14.633,108.0,...,1,187,<python><performance><for-loop><pandas>,What is the most efficient way to loop through...,195364.0,"Date,Open,High,Low,Close,Volume,Adj Close\r\n2...","Date,Open,High,Low,Close,Volume,Adj Close\r\n2...","Date,Open,High,Low,Close,Volume,Adj Close\r\n2...","2011-10-19,27.37,27.47,27.01,27.13,42880000,27...",NA
2,49,49,8916746.0,2.0,<p>I have a dataframe <code>df</code> in panda...,NaN,0,NaN,2012-01-18T19:41:27.017,12.0,...,1,26,<python><csv><numpy><tab-delimited><pandas>,selecting across multiple columns with python ...,19805.0,"df_greater_than10 = df[df[""colA""] > 10]\r\n","df_greater_than10 = df[df[""colA""] > 10]\r\n","df_greater_than10 = df[df[""colA""] > 10]\r\n","df_greater_than10 = df[df[""colA""] > 10]",NA
3,54,54,8997908.0,3.0,"<p>I recently came across the <a href=""http://...",NaN,16,NaN,2012-01-24T17:59:53.850,58.0,...,1,134,<python><r><join><data.table><pandas>,Why are pandas merges in python faster than da...,16173.0,,,,,NA
4,73,73,9557319.0,8.0,<p>I used Enthought's python distribution as a...,2013-12-04T20:55:59.357,5,NaN,2012-03-04T14:25:36.287,6.0,...,1,19,<python><numpy><scipy><enthought><pandas>,Open source Enthought Python alternative,6585.0,,,,,NA
5,80,80,9620832.0,2.0,"<p>I stumbled across <a href=""http://pandas.py...",NaN,2,NaN,2012-03-06T17:01:47.107,5.0,...,1,21,<python><pandas>,Simple cross-tabulation in pandas,9493.0,"AB,100.00\r\nAB,200.00\r\nAC,150.00\r\nAD,500....","AB,100.00\r\nAB,200.00\r\nAC,150.00\r\nAD,500....","AB,100.00\r\nAB,200.00\r\nAC,150.00\r\nAD,500....","AB,100.00\r\nAB,200.00\r\nAC,150.00\r\nAD,500....",NA
6,91,91,9652858.0,3.0,<p>I'm new to python and pandas. I'm trying t...,NaN,0,NaN,2012-03-11T06:00:56.347,3.0,...,1,38,<python><pandas><tsv>,How to I load a tsv file into a Pandas DataFrame?,32040.0,>>> df1 = DataFrame(csv.reader(open('c:/~/trai...,df1 = DataFrame(csv.reader(open('c:/~/trainSet...,df1 = DataFrame(csv.reader(open('c:/~/trainSet...,df1 = DataFrame(csv.reader(open('c:/~/trainSet...,NA
7,103,103,9762084.0,6.0,<p>I have manipulated some data using pandas a...,NaN,0,NaN,2012-03-18T12:53:06.683,13.0,...,1,34,<python><pandas>,Pandas convert dataframe to array of tuples,36433.0,In [182]: data_set\r\nOut[182]: \r\n index da...,In [182]: data_set\r\nOut[182]: \r\n index da...,data_set\r\nOut[182]: \r\n index data_date ...,data_set,NA
8,107,107,9772031.0,2.0,"<p>I'm a beginning pandas user, and after stud...",NaN,0,NaN,2012-03-18T22:34:26.183,2.0,...,1,11,<python><pandas>,Add indexed column to DataFrame with pandas,8189.0,df\r\n\r\n A ...,df\r\n\r\n A ...,df\r\n\r\n A ...,df\r\ndf2,NA
9,109,109,9794891.0,2.0,<p>I want to perform a join/merge/append opera...,NaN,0,NaN,2012-03-20T13:36:09.263,6.0,...,1,17,<python><pandas>,join or merge with overwrite in pandas,6054.0,,,,,NA


In [9]:
debug = True

solution_df.to_pickle(output_file)

if debug:
    solution_df.to_csv(output_file + ".csv", encoding='ISO-8859-1', sep=",", doublequote=True, index=False)

Error in method evaluate invalid syntax (<unknown>, line 1)
Error in method evaluate invalid syntax (<unknown>, line 1)
Error in method evaluate unexpected EOF while parsing (<unknown>, line 1)
Error in method evaluate invalid syntax (<unknown>, line 1)
Error in method evaluate invalid syntax (<unknown>, line 1)
Error in method evaluate unexpected EOF while parsing (<unknown>, line 1)
Error in method evaluate invalid syntax (<unknown>, line 1)
Error in method evaluate unexpected EOF while parsing (<unknown>, line 1)
Error in method evaluate invalid syntax (<unknown>, line 1)
Error in method evaluate invalid syntax (<unknown>, line 1)
Error in method evaluate invalid syntax (<unknown>, line 1)
Error in method evaluate unexpected indent (<unknown>, line 1)
Error in method evaluate invalid syntax (<unknown>, line 1)
Error in method evaluate unexpected EOF while parsing (<unknown>, line 1)
Error in method evaluate unexpected EOF while parsing (<unknown>, line 1)
Error in method evaluate un